In [1]:
def normalize(df_name, col_name):
    col_as_array = df_name[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.01, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

In [2]:
def prepfeatures(df_name):
    event = df_name['enc_event'].to_numpy()
    event = event.reshape(-1,1)
    
    duration = normalize(df_name,'duration')
    startTime = normalize(df_name,'UNIX_starttime')
    weekday = df_name['weekday'].to_numpy()
    
    prev_event = df_name['prev_event'].to_numpy()
    prev_event = prev_event.reshape(-1,1)
    prev_event = ordinal_encoder.fit_transform(prev_event)
    
    features = []
    for i in range(len(event)):
        current = event[i]
        current = np.append(current,duration[i])
        current = np.append(current,startTime[i])
        current = np.append(current,prev_event[i])
        current = np.append(current,weekday[i])
        features.append(current)
        
    return np.array(features)

In [ ]:
def preplabels(df_name):
    labels = df_name['next_event'].to_numpy()
    labels = label_encoder.fit_transform(labels)
    labels = labels.reshape(-1, 1)
    
    return np.array(labels)

In [ ]:
features = prepfeatures(train)
labels = preplabels(train)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(5,)),
    keras.layers.Dense(10, activation='softplus'),
    keras.layers.Dense(15, activation='softplus'),
    keras.layers.Dense(20, activation='softplus'),
    keras.layers.Dropout(1/20),
    keras.layers.Dense(25, activation='softplus'),
    keras.layers.Dense(30, activation='softplus'),
    keras.layers.Dropout(1/30),
    keras.layers.Dense(35, activation='softplus'),
    keras.layers.Dense(42, activation='softplus')
])

model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [3]:
model.fit(features,labels,epochs=5,verbose=1)

NameError: name 'model' is not defined

In [ ]:
features_test = prepfeatures(test)
labels_test = preplabels(test)

In [ ]:
test['neuralnet_event'] = model.predict(features_test)

In [ ]:
def prepfeatures_OLS(df_name):
    startTime = pd.to_datetime(df_name['startTime'])
    
    for i in range(len(startTime)):
        startTime[i] = startTime[i].timestamp()
        
    startTime = np.array(startTime).reshape(-1,1)
    
    event = df_name['event'].to_numpy()
    event = event.reshape(-1,1)
    event = ordinal_encoder.fit_transform(event)
    
    #payment_actual0 = normalize(df_name,'payment_actual0')
    #penalty_amount0 = normalize(df_name,'penalty_amount0')
    #number_parcels = normalize(df_name,'number_parcels')
    #area = normalize(df_name,'area')
    payment_actual0 = df_name['payment_actual0'].to_numpy()
    penalty_amount0 = df_name['penalty_amount0'].to_numpy()
    number_parcels = df_name['number_parcels'].to_numpy()
    area = df_name['area'].to_numpy()
    
    X = []
    for i in range(len(event)):
        current = startTime[i]
        current = np.append(current, event[i])
        current = np.append(current, payment_actual0[i])
        current = np.append(current, penalty_amount0[i])
        current = np.append(current, number_parcels[i])
        current = np.append(current, area[i])
        X.append(current)
        
    return np.array(X, dtype=float)

In [ ]:
def preplabels_OLS(df_name):
    duration = df_name['duration'].to_numpy()
    return np.array(duration, dtype=float)

In [ ]:
X = prepfeatures_OLS(train)

In [ ]:
y = preplabels_OLS(train)

In [ ]:
huber = HuberRegressor().fit(X, y)

In [ ]:
X_test = prepfeatures_OLS(test)

In [ ]:
test['regression_duration'] = huber.predict(X_test)

In [ ]:
test['error'] = np.absolute(test['duration'] - test['regression_duration'])

In [ ]:
test['error'].mean()